In [ ]:
import ccxt
import pandas as pd
import time
from datetime import datetime

exchange = ccxt.binance({
    'enableRateLimit': True,
})

symbols = ['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'LTC/USDT', 'ADA/USDT', 'XRP/USDT', 'SOL/USDT', 'DOGE/USDT']

timeframe = '1m'
limit = 1000
days = 70
minutes = days * 24 * 60 
chunks = minutes // limit + 1
end_time = exchange.milliseconds()

def fetch_ohlcv(symbol):
    print(f"Fetching: {symbol}")
    since = end_time - minutes * 60_000
    all_data = []

    for _ in range(chunks):
        try:
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit)
            if not ohlcv:
                break
            all_data.extend(ohlcv)
            since = ohlcv[-1][0] + 60_000
            time.sleep(1.1)
        except Exception as e:
            print(f"Error fetching {symbol}: {e}")
            break

    df = pd.DataFrame(all_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# def fetch_orderbook(symbol):
#     print(f"Fetching order book for: {symbol}")
#     since = 1745362800000
#     try:
#         orderbook = exchange.fetch_order_book(symbol, timeframe=timeframe, since = since, limit=limit)

for symbol in symbols:
    df = fetch_ohlcv(symbol)
    symbol_clean = symbol.replace('/', '')
    df.to_csv(f"{symbol_clean}_1m.csv", index=False)
    print(f"Saved {symbol_clean}_1m.csv")